In [3]:
from tensorflow import keras


model1 = keras.models.load_model('../models/pre_model_1_1_10_epochs.h5')
model2 = keras.models.load_model('../models/pre_model_2_1.h5')
model3 = keras.models.load_model('../models/pre_model_3_1_10_epochs.h5')
model4 = keras.models.load_model('../models/pre_model_4_1_10_epochs.h5')
model5 = keras.models.load_model('../models/pre_model_5_1_10_epochs.h5')
model6 = keras.models.load_model('../models/pre_model_6_1_20_2_epochs.h5')
model7 = keras.models.load_model('../models/pre_model_7_1_10_epochs.h5')
model8 = keras.models.load_model('../models/pre_model_8_1_10_epochs.h5')


In [16]:
all_models = [model1,model2, model3,model4,model5,model6,model7,model8]
rule_names = rules = [
            "Kein Scooter",
            "Scooter unvollständig/zu wenig Umgebung",
            "Scooter steht zu nah an Straße (Bordsteinkante)",
            "Scooter steht nicht frei",
            "Scooter auf Grünfläche",
            "Scooter blockiert Fahrradständer",
            "Scooter steht an ÖPNV Haltestelle / Parkverbo",
            "Scooter steht in Einfahrt / im Weg"
        ]

In [5]:
from PIL import Image
import numpy as np
# Function to load and resize an image
def load_image(img_path):
    with Image.open(img_path) as img:
        img = img.resize((512, 512))
        img = img.convert('RGB')  # Ensure image is in RGB format
        return np.array(img)


In [6]:
import matplotlib.pyplot as plt

def show_img(img_array):
    img_array.astype('uint8')
    plt.imshow(img_array)
    plt.axis('off')  # Hide the axes
    plt.show()

In [7]:
import os
import random

folder_path = "../Yoio_Park_Proof"
all_files = os.listdir(folder_path)
random_image_file = random.choice(all_files)
img_path = os.path.join(folder_path, random_image_file)
img = load_image(img_path)
img_array = np.expand_dims(img, axis=0)
print(img.shape)

(512, 512, 3)


In [8]:
import os
import random

folder_path = "../Yoio_Park_Proof"
all_files = os.listdir(folder_path)
random_image_file = random.choice(all_files)
img_path = os.path.join(folder_path, random_image_file)

def predict_image(models,img_path):


    img = load_image(img_path)

    for model in models:
        img_array = np.expand_dims(img, axis=0)
        predictions = model.predict(img_array)
        
        #Model 4 predicted Falsch herum i guess
        if models.index(model) !=5 :
            #if predictions[0][0] > predictions[0][1]: Wäre eine Option
            if predictions[0][0]>0.95:
                print("Verstoß")
                print(rules[models.index(model)])
                break
            else:
                print("Richtig")
        else: 
            if predictions[0][1]>0.95:
                print("Verstoß")
                print(rules[models.index(model)])
                break
            else:
                print("Richtig")


    show_img(img)
    

In [ ]:
#predict_image(models=all_models)

In [14]:
import pandas as pd
balanced_labels = pd.read_csv('../all_models_combine/chosen_labels_all.csv', sep=';',index_col=False)
balanced_labels.sample(5)

,Image_name,Rule1,Rule2,Rule3,Rule4,Rule5,Rule6,Rule7,Rule8
990,1607356715783ee43ab2b229eeda4f9f7862b57b415545...,0,0,0,0,0,1,0,0
376,1614717653d57d62bb471f9c1a00723a255038ca6fce5f...,1,0,0,0,0,0,0,0
982,1589901298fcaad67d86d1d5e0d50803dd87cef31591aa...,0,0,0,0,0,1,0,0
148,1585567100e84e2798118ae773a5a9242deef3691874f1...,0,0,0,0,0,0,0,0
756,16043335367e454ba3f0e3595e77a4bb1cf74fbbdd6d57...,0,1,0,0,0,1,0,0


In [20]:

columns=["Image_name","Rule1","Rule2","Rule3","Rule4","Rule5","Rule6","Rule7","Rule8"]
predictions_df = pd.DataFrame(columns=columns)

i=0
while i <= 1000:
    folder_path = "../Yoio_Park_Proof/"
    img_name=str(balanced_labels.sample(1).iloc[0,0])


    img = load_image(folder_path+img_name)

    predictions_array =[img_name]

    for model in all_models:
        img_array = np.expand_dims(img, axis=0)
        predictions = model.predict(img_array)
        #Model 4 predicted Falsch herum i guess
        if all_models.index(model) !=5 :
            #if predictions[0][0] > predictions[0][1]: Wäre eine Option
            if predictions[0][0]>0.95:
                int_pred=1
            else:
                int_pred=0
        else: 
            if predictions[0][1]>0.95:
                int_pred=1
            else:
                int_pred=0
        predictions_array.append(int_pred)

    predictions_df.loc[len(predictions_df)] = predictions_array
    #new_row = pd.DataFrame(predictions_array, columns=columns)
    #predictions_df = pd.concat([predictions_array,new_row],ignore_index=True)

    i += 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━

In [21]:
print(predictions_df.sample(10))

                                            Image_name  Rule1  Rule2  Rule3  \
780  15871011122a7dd18bc08081688ea4a8e41c599debda61...      0      1      0   
362  1614488866225bc53de468e99972267b28217e9b2e2698...      0      0      0   
41   1600616918d8658e141c69ed02726886a3c597c0f90a47...      0      1      0   
166  1586018743537c566904ce6908a148874e75f76ac18b58...      0      1      0   
266  1605250125175d918e61357f37dcd5e0dceec21004df0c...      0      1      0   
194  1614185862b174966c99498a2c7de6e6e3f8845e76cd63...      0      0      0   
602  160812471418cbedabdb42b74a59a4588178403e6f1d81...      0      1      0   
619  16014915510a463ba8a3afd50f049f7bd6519533db3f0a...      0      1      0   
683  1614665598ff3ce0e8033193b58731368a206c8dc385dd...      0      1      0   
761  1609476414651c719b01f6e8ef12d0851fcbcd4cc72c51...      0      1      0   

     Rule4  Rule5  Rule6  Rule7  Rule8  
780      0      0      0      0      0  
362      0      0      0      0      1  
41     

In [22]:
predictions_df.to_csv('../all_models_combine/predicted_labels_all.csv', sep=';', index=False)